# Question 2

**Quel était le classement des pilotes à l'issue de la saison 2023 ? Gérer les cas d'égalité (nombre de victoires, deuxièmes places, etc.).**

In [13]:
import pandas as pd

drivers = pd.read_csv('../data/drivers.csv')
results = pd.read_csv('../data/results.csv')
races = pd.read_csv('../data/races.csv')

results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,26520,1132,839,214,31,18,16,16,16,0.0,50,\N,\N,46,16,1:30.875,233.371,12
26515,26521,1132,815,9,11,0,17,17,17,0.0,50,\N,\N,50,6,1:29.707,236.409,12
26516,26522,1132,855,15,24,14,18,18,18,0.0,50,\N,\N,43,17,1:31.014,233.014,12
26517,26523,1132,847,131,63,1,\N,R,19,0.0,33,\N,\N,3,19,1:31.298,232.289,34


## Résolution

D'abord on regarde la 'positionOrder' (le rang final d'un pilote). Cependant il y a beaucoup d'égalités. On va également combiner avec le nombre de points.

Puisque l'on a le choix de la méthode de classement, je choisis de créer une colonne `rating` qui est une combinaison de la positionOrder et du nombre de points, et du rang. On va ensuite trier par `rating` pour obtenir le classement final.

In [ ]:
# 1. On ne récupère que les courses de 2023
results2023 = pd.merge(results,races,on = 'raceId').query('year == 2023')

results_join_drivers = pd.merge(drivers,results2023, on='driverId')[["driverId","forename", "surname", "positionOrder","points","rank"]].sort_values(by='points', ascending=False)

# 2. On groupe en sommant les points et en prenant la moyenne de positionOrder
grouped_results = results_join_drivers.groupby('driverId').agg({
    'points': 'sum',
    'positionOrder': 'mean',
    'forename': 'first',
    'surname': 'first',
    'rank': 'first'
}).reset_index()


# 3. On calcule le rating à l'aide de cette formule
grouped_results['rating'] = grouped_results['points'] - (grouped_results['positionOrder'] * 5) - (grouped_results['rank'].astype(int) * 10)
# grouped_results['rating'] = grouped_results['rating'].astype(int)

#4. On s'assure qu'il n'y a aucune égalité en vérifiant que le nombre de valeurs uniques est égal au nombre de lignes
assert grouped_results['rating'].nunique() == grouped_results['rating'].shape[0]

# 5. On affiche pour finir ⭐
grouped_results[["forename","surname","rating"]].sort_values(by='rating', ascending=False)

,forename,surname,rating
7,Max,Verstappen,513.636364
3,Sergio,Pérez,180.681818
0,Lewis,Hamilton,176.090909
1,Fernando,Alonso,154.818182
13,Lando,Norris,134.000000
8,Carlos,Sainz,70.500000
12,Charles,Leclerc,63.863636
14,George,Russell,46.545455
19,Oscar,Piastri,-2.545455
10,Lance,Stroll,-47.909091
